In [3]:
from CartAndPole.QLearningTabular.Agent import Agent
from CartAndPole.QLearningTabular.DoubleQLearningPolicy import DoubleQLearningPolicy as Policy
from Core.Utility.EpsilonGreedyStrategy import EpsilonGreedyStrategy
from CartAndPole.QLearningTabular.EnvironmentManager import EnvironmentManager
from IPython.display import clear_output, display
from itertools import count
from Core.Utility.Helper import Helper
from Core.Utility.DecayingLearningRate import DecayingLearningRate
from Core.Utility.UCB import UCB 
import numpy as np

epsilon_start = 1
epsilon_end = 0.01
epsilon_decay = 0.001

initial_lr = 0.01
decay_rate = 0.000002
min_lr = 0.001

In [ ]:



agent = Agent()
stateSpace = agent.initializeState()
epsilonGreedy = EpsilonGreedyStrategy(epsilon_start, epsilon_end, epsilon_decay)
decayingLearningRate = DecayingLearningRate(initial_lr, decay_rate, min_lr)
environment = EnvironmentManager('CartPole-v1')
policy1 = Policy() 
policy1.initializePolicy(stateSpace, environment.actionSpace.n)

policy2 = Policy() 
policy2.initializePolicy(stateSpace, environment.actionSpace.n)
ucb = UCB(environment.actionSpace.n, len(stateSpace), .1)



numberOfEpisodes = 500000
score = [0]
scoreCounter = 0
stateDict = dict(zip(stateSpace, range(len(stateSpace))))
for episode in range(numberOfEpisodes):
    stateHasChanged = True
        
    for step in count():
        
        
      
        if(stateHasChanged):
            
            currentState = agent.getState(environment.observation)
            # select = 1 if np.random.random() >= .5 else 0
            if(np.random.random() >= .5):
                selectionPolicy = policy1
                updatePolicy = policy2
            else:
                selectionPolicy = policy2
                updatePolicy = policy1
            
            action = epsilonGreedy.chooseAction(currentState, selectionPolicy, episode)
            # action = policy.getRandomAction()
            # action = policy.getMaxAction(currentState)
            # action = ucb.chooseAction(stateDict[currentState])
            reward, terminated = environment.step(action)
            # ucb.update(stateDict[currentState], action, reward)
            nextState = agent.getState(environment.observation)
         
           
            # policy.setTrainingRate(decayingLearningRate.getLearningRate(episode))
            if(not terminated):
                # policy.setTrainingRate(0.001)
                selectionPolicy.train(currentState, nextState, reward, action, updatePolicy)
            
            stateHasChanged = currentState != nextState
            
        else:
            reward, terminated = environment.step(action)
            nextState = agent.getState(environment.observation)
            stateHasChanged = currentState != nextState
          
        
        scoreCounter += 1
        
        
        
        if terminated:
            reward = -10
            selectionPolicy.train(currentState, nextState, reward, action, updatePolicy)
            
            score.append(scoreCounter)
            scoreCounter = 0
            environment.reset()
            
            if(episode % 200 == 0):
                clear_output(wait=True)
                display(f"episode {episode}")
                display(int(Helper.getMovingAverage3(300, score)))
                # utility.plot(300 ,score)
            
            break
        
       
            
            
environment.close()


In [11]:
environment = EnvironmentManager('CartPole-v1', True)

In [12]:
numberOfEpisodes = 15

for episode in range(numberOfEpisodes):
        
    for step in count():
        clear_output(wait=True)
        # display(f"step {episode}")
        currentState = agent.getState(environment.observation)
        
        action = policy1.getMaxAction(currentState)
        # action = policy.getRandomAction()
        reward, terminated = environment.step(action)
        nextState = agent.getState(environment.observation)
        
        
        
        if terminated:
            environment.reset()
            break
            
environment.close()

In [ ]:

s = []
for i, j in policy1.Q.items():
    if(max(j) > 0):
        s.append(f'{i} \t {j}')
        
print(len(s))

s
    


# import importlib
# ag = importlib.import_module("Utility")
# ag = importlib.reload(ag)


In [1]:
import gymnasium as gym
import numpy as np
env = gym.make('CartPole-v1')

state_low = env.observation_space.low
state_high = env.observation_space.high


n_bins = (4, 4, 4, 4)

tiler = gym.spaces.Box(low=state_low, high=state_high, dtype=np.float32)
    
print(tiler)
    


Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [ ]:
# Create Feature Vector
import numpy as np

cartPositionSpace = np.linspace(-2.4, 2.4, 10)
cartVelocitySpace = np.linspace(-4, 4, 5)
poleAngleSpace = np.linspace(-0.20943951, 0.20943951, 5)
poleAngleVelocitySpace = np.linspace(-4, 4, 5)



In [6]:
import numpy as np
cartPositionMin, cartPositionMax = -2.4, 2.4
cartVelocityMin, cartVelocityMax = -3.0, 3.0
poleAngleMin, poleAngleMax = -0.20943951, 0.20943951
poleAngularVelocityMin, poleAngularVelocityMax = -4.0, 4.0

numPositionBin = 8
numVelocityBin = 8
numAngleBin = 8
numAngularVelocityBin = 8

# cartPositionBin = np.linspace(0, 10,  11)
# print(cartPositionBin)

def tiles(cartPosition, cartVelocity, poleAngle, poleAngularVelocity):
    tileIndices = []
    numTilings = 3
    
    cartPositionBin = np.linspace(cartPositionMin, cartPositionMax, numPositionBin + 1)
    cartVelocityBin = np.linspace(cartVelocityMin, cartVelocityMax, numVelocityBin + 1)
    poleAngleBin = np.linspace(poleAngleMin, poleAngleMax, numAngleBin + 1)
    poleAngularVelocityBin = np.linspace(poleAngularVelocityMin, poleAngularVelocityMax, numAngularVelocityBin + 1)
    
    
    
    for tiling in range(numTilings):
        cartPositionOverLappingUnit = tiling * (cartPositionMax - cartPositionMin) / (numPositionBin * numTilings)
        cartVelocityOverLappingUnit = tiling * (cartVelocityMax - cartVelocityMin) / (numVelocityBin * numTilings)
        poleAngleOverLappingUnit = tiling * (poleAngleMax - poleAngleMin) / (numAngleBin * numTilings)
        poleAngularVelocityOverLappingUnit = tiling * (poleAngularVelocityMax - poleAngularVelocityMin) / (numAngularVelocityBin * numTilings)
        
        
        
        
        
        
        cartPositionIndex = np.digitize(cartPosition, cartPositionBin + cartPositionOverLappingUnit)
        cartVelocityIndex = np.digitize(cartVelocity, cartVelocityBin + cartVelocityOverLappingUnit)
        poleAngleIndex = np.digitize(poleAngle, poleAngleBin + poleAngleOverLappingUnit)
        poleAngularVelocityIndex = np.digitize(poleAngularVelocity, poleAngularVelocityBin + poleAngularVelocityOverLappingUnit)
        
        tileIndex = (tiling * (numPositionBin + numVelocityBin + numAngleBin + numAngularVelocityBin)
                     + cartPositionIndex 
                     + cartVelocityIndex * numPositionBin
                     + poleAngleIndex * numPositionBin * numVelocityBin
                     + poleAngularVelocityIndex * numAngleBin * numPositionBin * numVelocityBin
                     )
        
        tileIndices.append(tileIndex)
        
        return tileIndices
    
    numWeights = numTilings * (numVelocityBin + numPositionBin + numAngleBin + numAngularVelocityBin)
    weights = np.zeros(numWeights)
    
    featureVector = np.zeros(numWeights)
    
#     feature_vector = np.zeros(num_weights)  # Initialize feature vector
# for tile_index in active_tiles:
#     feature_vector[tile_index] = 1 
    
    
        
        
        

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [12]:


# np.array((numX, numY))
numX = 4
numY = 5
numZ = 3

def getUnit(x, y, z):
    return x + numX * y + z * numX * numY

for z in range(numZ):
    for y in range(numY):
        for x in range(numX):
        
            print(f'x : {x} \t y : {y} \t z : {z} \t unit : {getUnit(x, y, z)}')



x : 0 	 y : 0 	 z : 0
0
x : 1 	 y : 0 	 z : 0
1
x : 2 	 y : 0 	 z : 0
2
x : 3 	 y : 0 	 z : 0
3
x : 0 	 y : 1 	 z : 0
4
x : 1 	 y : 1 	 z : 0
5
x : 2 	 y : 1 	 z : 0
6
x : 3 	 y : 1 	 z : 0
7
x : 0 	 y : 2 	 z : 0
8
x : 1 	 y : 2 	 z : 0
9
x : 2 	 y : 2 	 z : 0
10
x : 3 	 y : 2 	 z : 0
11
x : 0 	 y : 3 	 z : 0
12
x : 1 	 y : 3 	 z : 0
13
x : 2 	 y : 3 	 z : 0
14
x : 3 	 y : 3 	 z : 0
15
x : 0 	 y : 4 	 z : 0
16
x : 1 	 y : 4 	 z : 0
17
x : 2 	 y : 4 	 z : 0
18
x : 3 	 y : 4 	 z : 0
19
x : 0 	 y : 0 	 z : 1
20
x : 1 	 y : 0 	 z : 1
21
x : 2 	 y : 0 	 z : 1
22
x : 3 	 y : 0 	 z : 1
23
x : 0 	 y : 1 	 z : 1
24
x : 1 	 y : 1 	 z : 1
25
x : 2 	 y : 1 	 z : 1
26
x : 3 	 y : 1 	 z : 1
27
x : 0 	 y : 2 	 z : 1
28
x : 1 	 y : 2 	 z : 1
29
x : 2 	 y : 2 	 z : 1
30
x : 3 	 y : 2 	 z : 1
31
x : 0 	 y : 3 	 z : 1
32
x : 1 	 y : 3 	 z : 1
33
x : 2 	 y : 3 	 z : 1
34
x : 3 	 y : 3 	 z : 1
35
x : 0 	 y : 4 	 z : 1
36
x : 1 	 y : 4 	 z : 1
37
x : 2 	 y : 4 	 z : 1
38
x : 3 	 y : 4 	 z : 1
39
x : 0 	 y 